## import de librerias

In [2]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

dataframes

In [3]:
df_users = pd.read_csv("recursos/users-details-2023.csv")
df_scores = pd.read_csv("recursos/users-score-2023.csv")
df_anime = pd.read_csv("recursos/anime-dataset-2023.csv")

In [4]:
# Función para calcular la edad a partir de la fecha de nacimiento
def calculate_age(birthday):
    try:
        birth_date = datetime.strptime(birthday.split('T')[0], '%Y-%m-%d')
        today = datetime.now()
        age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
        return age
    except:
        return None

In [5]:
df_users

,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.6,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.8,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0
3,9,Arcane,NaN,NaN,NaN,2004-12-05T00:00:00+00:00,30.0,7.71,5.0,54.0,4.0,3.0,0.0,66.0,0.0,1817.0
4,18,Mad,NaN,NaN,NaN,2005-01-03T00:00:00+00:00,52.0,6.27,1.0,114.0,10.0,5.0,23.0,153.0,42.0,3038.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731285,1291083,Dolopa,NaN,NaN,NaN,2012-05-06T00:00:00+00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731286,1291085,alenrobnik,NaN,NaN,NaN,2012-05-06T00:00:00+00:00,21.4,8.22,16.0,58.0,9.0,1.0,42.0,126.0,0.0,1239.0
731287,1291087,Oblongata,NaN,1993-01-30T00:00:00+00:00,NaN,2012-05-06T00:00:00+00:00,51.3,7.53,38.0,175.0,0.0,9.0,211.0,433.0,2.0,3010.0
731288,1291091,Etnota,NaN,NaN,NaN,2012-05-06T00:00:00+00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Procesar la edad
df_users['age'] = df_users['Birthday'].apply(calculate_age)

# Limpiar datos - eliminar edades no razonables y valores nulos
df_users = df_users[(df_users['age'] >= 10) & (df_users['age'] <= 80)]
df_users = df_users.dropna(subset=['Gender', 'age'])

In [8]:
df_users['watched_time'] = (df_users['Episodes Watched'] * 0.366666).round(2)

In [9]:
# Definir los bins (rangos) y etiquetas para grupos de 10 en 10
bins = range(10, 81, 5)  # De 10 a 80 en pasos de 5
labels = [f"{i}-{i+4}" for i in range(10, 80, 5)]  # Ej: "10-14", "15-19", etc.


# Crear una nueva columna con los rangos de edad
df_users['age_group'] = pd.cut(df_users['age'], bins=bins, labels=labels, right=False)

In [10]:
avg_watched_time = df_users.groupby('age_group')['watched_time'].mean().reset_index()
avg_watched_time.columns = ['age_group', 'avg_hours']

C:\Users\tomas\AppData\Local\Temp\ipykernel_17736\1779946765.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_watched_time = df_users.groupby('age_group')['watched_time'].mean().reset_index()


In [12]:
avg_watched_time = df_users.groupby('age_group')['watched_time'].mean().reset_index()

# Crear gráfico de lollipop verdadero
fig = go.Figure()

# Añadir las líneas (los "palos" del lollipop)
for i in range(len(avg_watched_time)):
    fig.add_shape(
        type='line',
        x0=avg_watched_time['age_group'][i],
        y0=0,
        x1=avg_watched_time['age_group'][i],
        y1=avg_watched_time['watched_time'][i],
        line=dict(color='rgb(102, 197, 204)', width=2)
    )

# Añadir los puntos (los "dulces" del lollipop)
fig.add_trace(go.Scatter(
    x=avg_watched_time['age_group'],
    y=avg_watched_time['watched_time'],
    mode='markers',
    marker=dict(color='rgb(102, 197, 204)', size=12),
    name='Horas vistas'
))

# Personalizar el gráfico
fig.update_layout(
    title='Minutos Vistas por Grupo de Edad',
    xaxis_title='Grupo de Edad',
    yaxis_title='Minutos Vistas Promedio',
    showlegend=False,
    template='plotly_white',
    yaxis=dict(range=[0, avg_watched_time['watched_time'].max() * 1.1])  # Ajustar el rango del eje Y
)

fig.show()

C:\Users\tomas\AppData\Local\Temp\ipykernel_17736\3719447173.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [10]:
# Preparar los datos para el gráfico de radar
radar_dropped = df_users.groupby('age_group')['Dropped'].mean().reset_index()

# Crear gráfico de radar
fig = px.line_polar(
    radar_dropped, 
    r='Dropped', 
    theta='age_group',
    line_close=True,
    title="Promedio de Animes Dropeados por Grupo de Edad",
    color_discrete_sequence=['rgb(220, 176, 242)']
)
fig.update_traces(fill='toself')
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, radar_dropped['Dropped'].max() * 1.2]  # Ajustar rango
        )),
    showlegend=False
)
fig.show()

C:\Users\tomas\AppData\Local\Temp\ipykernel_23704\1367770148.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

